In [ ]:
class Particula:
    
    def __init__(self, n_variables, limites_inf=None, limites_sup=None):
        # Inicializa la clase Particula con el número de variables y los límites inferiores y superiores
        self.n_variables = n_variables
        self.limites_inf = limites_inf
        self.limites_sup = limites_sup
        self.posicion = np.repeat(None, n_variables)  # Inicializa la posición con None
        self.velocidad = np.repeat(None, n_variables)  # Inicializa la velocidad con None
        self.valor = np.repeat(None, 1)  # Inicializa el valor con None
        self.mejor_valor = None  # Inicializa el mejor valor con None
        self.mejor_posicion = None  # Inicializa la mejor posición con None
        
        # Bucle para asignar un valor a cada una de las variables que definen la posición
        numbers = []
        remaining_sum = 1.0

        for i in range(self.n_variables - 1):
            # Define los valores máximo y mínimo para la variable actual
            max_value = min(0.25, remaining_sum - 0.01 * ((self.n_variables) - len(numbers) - 1))
            min_value = max(0.01, remaining_sum - 0.25 * ((self.n_variables) - len(numbers) - 1))
            # Genera un número aleatorio entre min_value y max_value
            number = random.uniform(min_value, max_value)
            numbers.append(number)
            self.posicion[i] = number
            remaining_sum -= number

        # Añadir el último número que complete la suma a 1
        last_number = round(remaining_sum, 10)
        if 0.01 <= last_number <= 0.25:
            numbers.append(last_number)
            self.posicion[-1] = last_number
        else:
            # Ajustar si el último número no está en el rango
            numbers[-1] += last_number - numbers[-1]
            self.posicion[-1] = last_number

        # La velocidad inicial de la partícula es 0
        self.velocidad = np.repeat(0, self.n_variables)

    def evaluar_particula(self, funcion_objetivo):
        # Evaluación de la función objetivo en la posición actual y siempre se maximiza el fitness score
        self.valor = funcion_objetivo(self.posicion)
        if self.mejor_valor is None:
            self.mejor_valor = np.copy(self.valor)
            self.mejor_posicion = np.copy(self.posicion)
       
        if self.valor > self.mejor_valor:
            self.mejor_valor = np.copy(self.valor)
            self.mejor_posicion = np.copy(self.posicion)
            
    def mover_particula(self, mejor_p_enjambre, inercia, peso_cognitivo, peso_social):
        # Actualización de la velocidad
        componente_velocidad = inercia * self.velocidad
        r1 = np.random.uniform(low=0.0, high=1.0, size=len(self.velocidad))
        r2 = np.random.uniform(low=0.0, high=1.0, size=len(self.velocidad))
        componente_cognitivo = peso_cognitivo * r1 * (self.mejor_posicion - self.posicion)
        componente_social = peso_social * r2 * (mejor_p_enjambre - self.posicion)
        nueva_velocidad = componente_velocidad + componente_cognitivo + componente_social
        self.velocidad = nueva_velocidad
        
        # Actualización de la posición
        self.posicion = self.posicion + self.velocidad
        suma_posicion = np.sum(self.posicion)
        if suma_posicion != 1:
            self.posicion = self.posicion / suma_posicion

        for i in np.arange(len(self.posicion)):
            if self.posicion[i] < self.limites_inf[i]:
                self.posicion[i] = self.limites_inf[i]
                self.velocidad[i] = 0

            if self.posicion[i] > self.limites_sup[i]:
                self.posicion[i] = self.limites_sup[i]
                self.velocidad[i] = 0
                
################################################################################
#                              CLASE ENJAMBRE (SWARM)                          #
################################################################################

class Enjambre:

    def __init__(self, n_particulas, n_variables, limites_inf=None, limites_sup=None):
        # Inicializa la clase Enjambre con el número de partículas, variables y sus límites
        self.n_particulas = n_particulas
        self.n_variables = n_variables
        self.limites_inf = limites_inf
        self.limites_sup = limites_sup
        self.particulas = []
        self.optimizado = False  # Etiqueta para saber si el enjambre ha sido optimizado
        self.iter_optimizacion = None  # Número de iteraciones de optimización llevadas a cabo
        self.mejor_particula = None
        self.mejor_valor = None
        self.mejor_posicion = None
        self.historico_particulas = []
        self.historico_mejor_posicion = []
        self.historico_mejor_valor = []
        self.diferencia_abs = []
        self.resultados_df = None
        self.iteracion_mejor_valor = []
        self.iteracion_optima = None
        self.valor_optimo = None
        self.posicion_optima = None

        # Se crean las partículas del enjambre y se almacenan
        for i in np.arange(n_particulas):
            particula_i = Particula(
                n_variables=self.n_variables,
                limites_inf=self.limites_inf,
                limites_sup=self.limites_sup
            )
            self.particulas.append(particula_i)

    def mostrar_particulas(self, n=None):
        # Muestra las partículas del enjambre
        if n is None:
            n = self.n_particulas
        elif n > self.n_particulas:
            n = self.n_particulas

        return None

    def evaluar_enjambre(self, funcion_objetivo):
        # Se evalúa cada partícula del enjambre
        for i in np.arange(self.n_particulas):
            self.particulas[i].evaluar_particula(
                funcion_objetivo=funcion_objetivo
            )

        # Mejor partícula del enjambre
        self.mejor_particula = copy.deepcopy(self.particulas[0])
        for i in np.arange(self.n_particulas):
            if self.particulas[i].valor > self.mejor_particula.valor:
                self.mejor_particula = copy.deepcopy(self.particulas[i])

        # Se extrae la posición y valor de la mejor partícula 
        self.mejor_valor = self.mejor_particula.valor
        self.mejor_posicion = self.mejor_particula.posicion
        
    def mover_enjambre(self, inercia, peso_cognitivo, peso_social):
        """
        Este método mueve todas las partículas del enjambre.
        """
        for i in np.arange(self.n_particulas):
            self.particulas[i].mover_particula(
                mejor_p_enjambre=self.mejor_posicion,
                inercia=inercia,
                peso_cognitivo=peso_cognitivo,
                peso_social=peso_social
            )

    def optimizar(self, funcion_objetivo, n_iteraciones, inercia, reduc_inercia, inercia_max, inercia_min, peso_cognitivo, peso_social, parada_temprana, rondas_parada, tolerancia_parada):
        # Comprobaciones iniciales: exceptions y warnings
        if reduc_inercia and (inercia_max is None or inercia_min is None):
            raise Exception(
                "Para activar la reducción de inercia es necesario indicar un valor de inercia_max y de inercia_min."
            )

        # Iteraciones
        for i in np.arange(n_iteraciones):
            print("-------------")
            print("Iteracion: " + str(i))
            print("-------------")
            
            # Evaluar partículas del enjambre
            self.evaluar_enjambre(
                funcion_objetivo=funcion_objetivo
            )

            # Se almacena la información de la iteración en los históricos
            self.historico_particulas.append(copy.deepcopy(self.particulas))
            self.historico_mejor_posicion.append(copy.deepcopy(self.mejor_posicion))
            self.historico_mejor_valor.append(copy.deepcopy(self.mejor_valor))
            self.iteracion_mejor_valor.append(i)

            # Se calcula la diferencia absoluta respecto a la iteración anterior
            if i == 0:
                self.diferencia_abs.append(None)
            else:
                diferencia = abs(self.historico_mejor_valor[i] - self.historico_mejor_valor[i - 1])
                self.diferencia_abs.append(diferencia)

            # Mover partículas del enjambre
            if reduc_inercia:
                inercia = ((inercia_max - inercia_min) * (n_iteraciones - i) / n_iteraciones) + inercia_min
           
            self.mover_enjambre(
                inercia=inercia,
                peso_cognitivo=peso_cognitivo,
                peso_social=peso_social
            )

        self.optimizado = True
        self.iter_optimizacion = i
        
        # Identificación del mejor individuo de todo el proceso
        print('vector', self.historico_mejor_valor)
        for _ in range(len(self.historico_mejor_valor)):
            indice